> THIS NOTEBOOK SHOULD BE UPDATED. THERE IS A LOT OF MISSING THINGS

# Machine learning for Time Series Data

The dataset we are going to use comes from:
- Heartbeat wave: https://www.kaggle.com/datasets/kinguistics/heartbeat-sounds/
- Stock prices

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from glob import glob
import librosa as lr 

audio, sfreq = lr.load('../data/murmur__201101051104.wav')

In [ ]:
print(sfreq)

Knowing the sampling rate of a timeseries we can know the timestamp of each datapoint relative to the first datapoint.

(assuming the sampling rate is fixed and no datapoints are lost)

In [ ]:
audio

In [ ]:
len(audio)

In [ ]:
indices = np.arange(0, len(audio))
time = indices/sfreq

In [ ]:
plt.plot(time, audio)

In [ ]:
prices = pd.read_csv('../data/prices.csv', parse_dates=['date'], index_col='date')
prices

In [ ]:
pivot_prices = prices.pivot(values='close', columns='symbol')
pivot_prices = pivot_prices[['AAPL', 'FB', 'NFLX', 'V']]

In [ ]:
# Loop through each column, plot its values over time
fig, ax = plt.subplots()
for column in pivot_prices.columns:
    pivot_prices[column].plot(ax=ax, label=column)
ax.legend()
plt.show()

In [ ]:
glob("../data/heartbeat/set_a/*")

# Classification

A first version can be to try to predict each class based on the max, std and mean of each signal.

## improving the features used for classification

### AudioEnvelope

Smooth the data to calculate the auditory envelope

Related to the total amount of audio energy present at each moment of time (rectifying the signal, that is, taking the absolute value of each point in time)

Performing a local average, smoothing the timeseries (rolling window), removes short term noise while retaining the general pattern

### The Tempogram

We can summarize more complex temporal information with timeseries specific functions

Libros is a great library for auditoyu and imeseries feature engineering

The tempogram estimates the tempo of a sound over time

We can calculate summary statistics of tempo in the same way that we can for the envelope




In [ ]:
audio_tempo = lr.beat.tempo(audio, sr=sfreq, hop_length=2**6, aggregate=None) 

### The spectogram 

Timeseries data can be described as a combination of quickly-changing things and slowly-
changing things

At each moment in time, we can describe the relative presence of fast- and slow-moving components

The simplest way to do this is called a Fourier Transform

This converts a single timeseries into an array that describes the timeseries as a
combination of oscillations

We can calculate the STFT with librosa

There are several parameters we can tweak 

we can visualize the spectogram with the specshow function



In [ ]:
# Import the functions we'll use for the STFT
from librosa.core import stft, amplitude_to_db from librosa.display import specshow import matplotlib. pyplot as plt

# Calculate our STFT
HOP_LENGTH = 2**4
SIZE_WINDOW = 2**7
audio_spec = stft(audio, hop_Length=HOP_LENGTH, n_fft=SIZE_WINDOW)

# Convert into decibels for visualization
spec_db = amplitude_to_db(audio_spec)

# Visualize
fig, ax = plt.subplots)
specshow(spec_db, sp=sfrea, x_axis='time',

## Dealing with Real World Data

When dealing with real data we can face gaps in the time series.

One usual solution for this is to **interpolate** in the gaps.



In [ ]:
pivot_prices.loc['2012':'2013', 'AAPL']=np.nan

In [ ]:
pivot_prices.plot()

In [ ]:
pivot_prices.interpolate('linear').plot()

We can use rolling windows to transform the data by smoothing it.

Rolling windows can be used to do more complex transformations.

To standardize the variance of a time series we can calculate the % change over a previous window

This makes timepoints more comparable to one another if the absolute values of data change a lot 


In [ ]:
def percent_change(values) :
    """Calculates the % change between the last value and the mean of previous values"""
    # Separate the last value and all previous values into variables
    previous_values = values[:-1]
    last_value = values[-1]
    # Calculate the % difference between the last value
    # and the mean of earlier values
    
    percent_change = (last_value - np.mean (previous_values)) \
        / np. mean (previous_values)
    return percent_change

In [ ]:
pivot_prices.NFLX.plot()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10, 5))
ax = pivot_prices.NFLX.plot(ax = axs[0])

ax = pivot_prices.NFLX.rolling(window=20).aggregate(percent_change).plot(ax = axs[1])


In [ ]:
This transformation can be used to identify outliers.

An outlier is a value that is 3 std away from the mean.

We can replace the outliers by the median

# Creating Features from the past. 

> A common question to ask when dealing with timeseries is: how smooth is the data? That is how correlated is a timepoint with its neighboring timepoints (autocorrelation). The amount of autocorrelation will impact your models

When using cross validation its important to avoid shuffling the data because that would break its temporal structure.

SKlearn has the timeSeriesSplit that prevents this issue, ensuring we always use past data to predict the future.

# Stationarity

Stationary time series do not change their statistical properties over time (mean, std..)

The stationarity of our data has an impact on the models: non stationary data results in variability in our model.

The statistical properties the model find may change with the data. How can we quantify this? 

One approach is to use cross validation and assess parameter stability across all cv splits.

Bootstrapping the mean is another way to assess the stationarity of a timeseries.